![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**0- Connect to your Google Drive folder**

In [86]:
my_folder = "/content/drive/MyDrive/retoSAS"

from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/retoSAS


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [87]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [88]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [89]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [90]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
#'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

If there is no .authinfo file, you can create this

In [91]:
#%%writefile .authinfo
#oda user  password 

Copy this file to home

In [92]:
!cp .authinfo ~/.authinfo

**5- Establish Connection (Need to do this step each time you use SASPy)**

In [93]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 636



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/retoSAS/sascfg_personal.py
WORK Path             = /saswork/SAS_workE4170000A500_odaws01-usw2-2.oda.sas.com/SAS_work51BA0000A500_odaws01-usw2-2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 42240


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [94]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 05:14:00 PM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 05:14:00 PM

33         


For local Jupyter

In [95]:
#%%SAS sas_session
#libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [96]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [97]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
831011,2831012.0,BERNARD,CLARK,0.0,26.0,31000.0,High School,City,3.0,1.0,...,1.0,1.0,90.0,90.0,90.0,0.0,0.0,0.0,1.0,40.0
986164,2986165.0,BENSON,TARA,1.0,20.0,52200.0,University / College,City,6.0,0.0,...,1.0,5.0,30.0,30.0,30.0,0.0,0.0,0.0,1.0,40.0


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [98]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)  
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)  


data_merge.sample(5)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
86967,2086968.0,SMITH,LEON,0.0,21.0,69400.0,University / College,Suburban,3.0,0.0,...,1.0,3.0,10.0,10.0,10.0,0.0,0.0,0.0,1.0,20.0
624611,2624612.0,MAXWELL,JACQUELYN,1.0,60.0,16800.0,High School,City,8.0,0.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,1.0,10.0
443318,2443319.0,BANKS,PEDRO,0.0,33.0,26500.0,High School,City,0.0,0.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,1.0,10.0
15617,2015618.0,JONES,ESTER,1.0,71.0,15700.0,University / College,Suburban,3.0,1.0,...,1.0,1.0,10.0,10.0,10.0,0.0,0.0,0.0,1.0,10.0
603883,2603884.0,ALICEA,DARREN,0.0,28.0,31800.0,University / College,City,8.0,1.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,1.0,20.0


## Data Partition

In [99]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.5,random_state=5678) # you can change the percentage
train.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
639370,2639371.0,LAWRENCE,TENISHA,1.0,16.0,27400.0,University / College,Downtown,10.0,2.0,...,5.0,10.0,190.0,10.0,100.0,0.0,0.0,1.0,1.0,20.0
141089,2141090.0,MOUTREY,ANDREA,0.0,42.0,58300.0,University / College,Suburban,7.0,1.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,1.0,1.0,10.0
616723,2616724.0,DOVE,BETH,1.0,29.0,51100.0,High School,City,1.0,0.0,...,1.0,1.0,40.0,40.0,40.0,0.0,0.0,0.0,1.0,800.0
440950,2440951.0,STEELE,ERIC,0.0,24.0,6700.0,University / College,Downtown,5.0,0.0,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,1.0,30.0
140037,2140038.0,PIERSALL,ANNA,1.0,24.0,16100.0,University / College,Downtown,8.0,0.0,...,1.0,1.0,30.0,30.0,30.0,0.0,0.0,0.0,1.0,30.0



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [100]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[ 51.16213414  13.54362045 120.78262404 ...  37.6595094   68.71680603
  60.2614866 ]


In [101]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

63.977629882676396
54397.08072509484
233.23181756590338


## Regression Tree Model（Py）

In [102]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=4, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 51.88865666  30.23313158 169.85860826 ...  53.57906847  48.42181748
  68.75937647]


In [103]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

63.18951052949899
53968.353579257775
232.31089853740778


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [116]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [118]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_contact[['MinGift']] = scoring_data_contact[['MinGift']].fillna(value=0) 

#scoring_data_contact.head()

X = scoring_data_contact[['Age', 'Salary','Contact','MinGift','GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 

#regr_predict_contact=regr.predict(X)
XGB_predict_contact=XGB_model.predict(X)

#scoring_data_contact['Prediction'] = regr_predict_contact
scoring_data_contact['Prediction'] = XGB_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,62.043304
1,2000002.0,57.124481
2,2000003.0,101.851738
3,2000004.0,44.299194
4,2000005.0,103.550179


### Predict 'amount given' for members who were not contacted

In [120]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['MinGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)
scoring_data_nocontact[['TotalGift']] = scoring_data_nocontact[['TotalGift']].fillna(value=0)


#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 

#regr_predict_nocontact=regr.predict(X)
XGB_predict_nocontact=XGB_model.predict(X)

#scoring_data_nocontact['Prediction'] = regr_predict_contact
scoring_data_nocontact['Prediction'] = XGB_predict_contact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,62.043304
1,2000002.0,57.124481
2,2000003.0,101.851738
3,2000004.0,44.299194
4,2000005.0,103.550179


In [122]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,62.043304,62.043304
1,2000002.0,57.124481,57.124481
2,2000003.0,101.851738,101.851738


## Exporting Results to a CSV File

In [123]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [124]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.

In [125]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,62.043304,62.043304
2000002.0,57.12448,57.12448
2000003.0,101.85174,101.85174
2000004.0,44.299194,44.299194
2000005.0,103.55018,103.55018
2000006.0,35.58959,35.58959
2000007.0,29.467974,29.467974
2000008.0,53.878136,53.878136
2000009.0,42.36442,42.36442
